In [30]:
!pip install git+https://github.com/openai/whisper.git pydub


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tl5yeyzp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tl5yeyzp
  Resolved https://github.com/openai/whisper.git to commit 5979f03701209bb035a0a466f14131aeb1116cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [40]:

import whisper
from pydub import AudioSegment
from IPython.display import display, FileLink, Audio
from ipywidgets import FileUpload
import tempfile


model = whisper.load_model('base')


cuss_words = ["fuck", "damn", "shit", "bitch", "bullshit"]

def censor_audio(audio_path, transcription):
    audio = AudioSegment.from_file(audio_path)

    for segment in transcription['segments']:
        for word_info in segment['words']:
            word = word_info['word'].lower().strip()
            if word in cuss_words:

                start_time = word_info['start'] * 1000
                end_time = word_info['end'] * 1000


                audio = audio[:start_time] + AudioSegment.silent(duration=(end_time - start_time)) + audio[end_time:]


    censored_audio_path = "/content/censored_audio.mp3"
    audio.export(censored_audio_path, format="mp3")

    return censored_audio_path

def process_audio(uploaded_file):

    temp_audio_path = "/content/temp_audio.mp3"
    with open(temp_audio_path, "wb") as f:
        f.write(uploaded_file['content'])


    transcription = model.transcribe(temp_audio_path, language='en', word_timestamps=True)


    censored_audio_path = censor_audio(temp_audio_path, transcription)

    return censored_audio_path

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [28]:
uploader = FileUpload(accept='.wav,.mp3', multiple=False)
display(uploader)

FileUpload(value={}, accept='.wav,.mp3', description='Upload')

In [41]:
if uploader.value:
    uploaded_file = list(uploader.value.values())[0]
    refined_audio_path = process_audio(uploaded_file)


    print("Download your refined audio file:")
    display(FileLink(refined_audio_path))

    print("Play the refined audio below:")
    display(Audio(refined_audio_path, autoplay=True))

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Download your refined audio file:


/content/censored_audio.mp3

Play the refined audio below:
